In [1]:
REALTIME_ESTIMATOR = False
# set the window of blocks, will be overwritten if REALTIME_ESTIMATOR == True
week_1_start_ts = 1590969600
WEEK = 57
week_end_timestamp = week_1_start_ts + WEEK * 7 * 24 * 60 * 60
week_start_timestamp = week_end_timestamp - 7 * 24 * 60 * 60

In [2]:
incentivized_polygon_pools = [
    '0x0297e37f1873d2dab4487aa67cd56b58e2f27875000100000000000000000002',
    '0xce66904b68f1f070332cbc631de7ee98b650b499000100000000000000000009',
    '0x36128d5436d2d70cab39c9af9cce146c38554ff0000100000000000000000008',
    '0xf461f2240b66d55dcf9059e26c022160c06863bf000100000000000000000006',
    '0x32fc95287b14eaef3afa92cccc48c285ee3a280a000100000000000000000005'
]
incentivized_polygon_pools_addresses = [
    p[:42] for p in incentivized_polygon_pools
]

In [3]:
from google.cloud import bigquery
from google.cloud import bigquery_storage
import warnings
import time
from web3 import Web3
import pandas as pd


if REALTIME_ESTIMATOR:
    warnings.warn('Running realtime estimator')
    
    from urllib.request import urlopen
    import json
    url = 'https://ipfs.fleek.co/ipns/balancer-team-bucket.storage.fleek.co/balancer-claim/snapshot'
    jsonurl = urlopen(url)
    claims = json.loads(jsonurl.read())
    claimable_weeks = [20+int(w) for w in claims.keys()]
    most_recent_week = max(claimable_weeks)
    # delete the estimates for the most recent published week, since now there's an official value available on IPFS
    project_id = os.environ['GCP_PROJECT']
    sql = f'''
        DELETE FROM {project_id}.bal_mining_estimates.lp_estimates_tmp
        WHERE week = {most_recent_week}
    '''
    client = bigquery.Client()
    query = client.query(sql)
    query.result()
    
    
    from datetime import datetime
    week_1_start = '01/06/2020 00:00:00 UTC'
    week_1_start = datetime.strptime(week_1_start, '%d/%m/%Y %H:%M:%S %Z')
    WEEK = int(1 + (datetime.utcnow() - week_1_start).days/7)  # this is what week we're actually in
    week_end_timestamp = week_1_start_ts + WEEK * 7 * 24 * 60 * 60
    week_start_timestamp = week_end_timestamp - 7 * 24 * 60 * 60
    week_end_timestamp = int(datetime.utcnow().timestamp())
    week_passed = (week_end_timestamp - week_start_timestamp)/(7*24*3600)

Consider installing rusty-rlp to improve pyrlp performance with a rust based backend
<ipython-input-3-26c0cad345e5>:10: UserWarning: Running realtime estimator
  warnings.warn('Running realtime estimator')


In [4]:
from urllib.request import urlopen
import json
# V2 allocation
V2_LM_ALLOCATION_URL = 'https://raw.githubusercontent.com/balancer-labs/frontend-v2/master/src/lib/utils/liquidityMining/LiquidityMiningV2.json'
jsonurl = urlopen(V2_LM_ALLOCATION_URL)
try:
    V2_ALLOCATION_THIS_WEEK = json.loads(jsonurl.read())[f'week_{WEEK}']['tiers']
except KeyError:
    V2_ALLOCATION_THIS_WEEK = {}
V2_MINING_POOLS = {}
for tier in V2_ALLOCATION_THIS_WEEK.values():
    for pool in tier['slots'].values():
        pool_address = pool[:42].lower()
        V2_MINING_POOLS[pool_address] = V2_MINING_POOLS.get(pool_address,0) + tier['BAL']
BAL_MINED_ON_V2 = sum(V2_MINING_POOLS.values())
print(f'BAL mined on V2 on week {WEEK}: {BAL_MINED_ON_V2}')

CLAIM_PRECISION = 18 # leave out of results addresses that mined less than CLAIM_THRESHOLD BAL
CLAIM_THRESHOLD = 10**(-CLAIM_PRECISION)
reports_dir = f'reports/{WEEK}'

BAL mined on V2 on week 57: 145000


In [5]:
# get addresses that redirect
if REALTIME_ESTIMATOR:
    url = 'https://raw.githubusercontent.com/balancer-labs/bal-mining-scripts/master/config/redirect.json'
    jsonurl = urlopen(url)
    redirects = json.loads(jsonurl.read())
else:
    redirects = json.load(open('config/redirect.json'))

# V2 Liquidity Mining

In [6]:
def v2_liquidity_mining(week, 
                        pools_addresses_and_BAL_earned,
                        network):
    print(f'-----------------{network}-----------------')

    network_blocks_table = {
        'ethereum': 'bigquery-public-data.crypto_ethereum.blocks',
        'polygon': 'public-data-finance.crypto_polygon.blocks',
    }
    
    bpt_balances_table = {
        'ethereum': 'blockchain-etl.ethereum_balancer.view_token_balances_subset',
        'polygon': 'blockchain-etl.polygon_balancer.view_bpt_balances',
    }

    with open('src/liquidity_mining_V2.sql','r') as file:
        sql = (
            file
            .read()
            .format(
                week, 
                '\',\''.join(pools_addresses_and_BAL_earned.keys()),
                network_blocks_table[network],
                bpt_balances_table[network]
            )
        )
    # print(sql)

    print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) + ' - Querying Bigquery for the V2 LPs...')

    client = bigquery.Client()
    bqstorageclient = bigquery_storage.BigQueryReadClient()
    time_weighted_share = (
        client.query(sql)
        .result()
        .to_dataframe(bqstorage_client=bqstorageclient)
    )
    print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) + ' - Done!')
    time_weighted_share['miner'] = time_weighted_share['miner'].apply(Web3.toChecksumAddress)
    time_weighted_share.set_index(['pool_address','miner'], inplace=True)

    weekly_pools_BAL = pd.Series(pools_addresses_and_BAL_earned)
    weekly_pools_BAL.index.name = 'pool_address'

    bal_mined_v2 = time_weighted_share['share']*weekly_pools_BAL
    if REALTIME_ESTIMATOR:
        bal_mined_v2 *= week_passed

    miner_export_v2 = bal_mined_v2.groupby('miner').sum()
    print(f'{miner_export_v2.sum()} BAL mined on {network}')

    v2_miners = pd.DataFrame(miner_export_v2).reset_index()
    n = len(v2_miners['miner'].drop_duplicates()[v2_miners['miner'].drop_duplicates().isin(redirects.keys())])
    print(f'Redirect: {n} redirectors found')
    v2_miners['miner'] = v2_miners['miner'].apply(lambda x: redirects.get(x,x))
    miner_export_v2 = v2_miners.groupby('miner').sum()[0]

    if not REALTIME_ESTIMATOR:
        filename = f'/_{network}.json'
        (
            miner_export_v2[miner_export_v2>=CLAIM_THRESHOLD]
            .apply(
                lambda x: format(
                    x, 
                    f'.{CLAIM_PRECISION}f'
                )
            )
            .to_json(reports_dir+filename, indent=4)
        )

    return miner_export_v2


miners_ethereum = v2_liquidity_mining(
    WEEK, 
    {k: v for k,v in V2_MINING_POOLS.items() if k not in incentivized_polygon_pools_addresses},
    'ethereum')
miners_polygon = v2_liquidity_mining(
    WEEK, 
    {k: v for k,v in V2_MINING_POOLS.items() if k in incentivized_polygon_pools_addresses},
    'polygon')

-----------------ethereum-----------------
2021-06-29 19:16:33 - Querying Bigquery for the V2 LPs...
2021-06-29 19:17:15 - Done!
36223.60367063491 BAL mined on ethereum
Redirect: 0 redirectors found
-----------------polygon-----------------
2021-06-29 19:17:15 - Querying Bigquery for the V2 LPs...
2021-06-29 19:17:52 - Done!
6306.133432539699 BAL mined on polygon
Redirect: 0 redirectors found


In [7]:
miners_all = miners_ethereum.add(miners_polygon, fill_value=0)
if not REALTIME_ESTIMATOR:
    filename = '/_totalsLiquidityMining.json'
    (
        miners_all[miners_all>=CLAIM_THRESHOLD]
        .apply(
            lambda x: format(
                x, 
                f'.{CLAIM_PRECISION}f'
            )
        )
        .to_json(reports_dir+filename, indent=4)
    )

# Update real time estimates in GBQ

In [8]:
if REALTIME_ESTIMATOR:
    sql = f'''
        UPDATE {project_id}.bal_mining_estimates.lp_estimates_tmp
        SET velocity = '0'
        WHERE week = {WEEK-1}
    '''
    client = bigquery.Client()
    query = client.query(sql)
    query.result();

    # write to GBQ (LPs)
    cur_estimate = pd.DataFrame(miners_ethereum)
    cur_estimate.columns = ['earned']
    cur_estimate.index.name = 'address'
    
    try:
        prev_estimate = pd.read_gbq(f'select address, earned, timestamp from bal_mining_estimates.lp_estimates_tmp WHERE week = {WEEK}', 
                        project_id=os.environ['GCP_PROJECT'])
        prev_estimate.set_index('address', inplace=True)
        prev_estimate_timestamp = prev_estimate.iloc[0]['timestamp']
    except:
        prev_estimate_timestamp = 0
    if prev_estimate_timestamp < week_start_timestamp:
        #previous estimate is last week's; compute velocity from end_block_timestamp and start_block_timestamp
        delta_t = (week_end_timestamp - week_start_timestamp)
        earned = cur_estimate['earned'].astype(float)
        cur_estimate['velocity'] = (earned/delta_t).apply(lambda x: format(x, f'.{18}f'))
    else:
        #compute velocity based on increase and time passed
        delta_t = (week_end_timestamp - prev_estimate_timestamp)
        diff_estimate = cur_estimate.join(prev_estimate, rsuffix='_prev').fillna(0)
        cur_earned = diff_estimate['earned'].astype(float)
        prev_earned = diff_estimate['earned_prev'].astype(float)
        cur_estimate['velocity'] = ((cur_earned-prev_earned)/delta_t).apply(lambda x: format(x, f'.{18}f'))
        
    # delete this week's previous estimates
    sql = f'''
        DELETE FROM {project_id}.bal_mining_estimates.lp_estimates_tmp
        WHERE week = {WEEK}
    '''
    client = bigquery.Client()
    query = client.query(sql)
    query.result();

    cur_estimate['earned'] = cur_estimate['earned'].apply(lambda x: format(x, f'.{18}f'))
    cur_estimate['timestamp'] = week_end_timestamp
    cur_estimate['week'] = WEEK
    cur_estimate.reset_index(inplace=True)
    cur_estimate.to_gbq( 'bal_mining_estimates.lp_estimates_tmp', 
                        project_id=os.environ['GCP_PROJECT'], 
                        if_exists='append')

1it [00:06,  6.19s/it]


# Gas Reimbursement Program

In [9]:
from src.bal4gas_V1 import compute_bal_for_gas as compute_bal_for_gas_V1
from src.bal4gas_V2 import compute_bal_for_gas as compute_bal_for_gas_V2

if not REALTIME_ESTIMATOR:
    whitelist = pd.read_json(f'https://raw.githubusercontent.com/balancer-labs/assets/w{WEEK-1}/lists/eligible.json').index.values
    gas_whitelist = pd.Series(whitelist).str.lower().tolist()
    gas_whitelist.append('0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee')

    
    v1 = compute_bal_for_gas_V1(week_start_timestamp, week_end_timestamp, gas_whitelist, plot=True, verbose=True)

    gas_whitelist.remove('0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee')
    gas_whitelist.append('0x0000000000000000000000000000000000000000')
    v2 = compute_bal_for_gas_V2(week_start_timestamp, week_end_timestamp, gas_whitelist, plot=True, verbose=True)
    
    merge = v1.append(v2)

    totals_bal4gas = merge[['address','bal_reimbursement']].groupby('address').sum()['bal_reimbursement']
    totals_bal4gas[totals_bal4gas>=CLAIM_THRESHOLD].apply(\
       lambda x: format(x, f'.{CLAIM_PRECISION}f')).to_json(reports_dir+'/_gasReimbursement.json',
       indent=4)

    # combine BAL from liquidity mining and gas reimbursements
    totals = miners_ethereum.add(totals_bal4gas, fill_value=0)
    totals[totals>=CLAIM_THRESHOLD].apply(\
       lambda x: format(x, f'.{CLAIM_PRECISION}f')).to_json(reports_dir+'/_totals.json',
       indent=4)

In [10]:
if not REALTIME_ESTIMATOR:
    print('Final Check Totals')
    _ethereum = pd.read_json(reports_dir+'/_ethereum.json', orient='index').sum().values[0]
    _polygon = pd.read_json(reports_dir+'/_polygon.json', orient='index').sum().values[0]
    _lm_both = pd.read_json(reports_dir+'/_totalsLiquidityMining.json', orient='index').sum().values[0]
    _claim = pd.read_json(reports_dir+'/_totals.json', orient='index').sum().values[0]
    print(f'Liquidity Mining Ethereum: {format(_ethereum, f".{CLAIM_PRECISION}f")}')
    print(f'Liquidity Mining Polygon: {format(_polygon, f".{CLAIM_PRECISION}f")}')
    print(f'Liquidity Mining Both: {format(_lm_both, f".{CLAIM_PRECISION}f")}')
    print(f'Gas Reimbursement week {WEEK}: {format(_claim-145000, f".{CLAIM_PRECISION}f")}')
    print(f'Claims: {format(_claim, f".{CLAIM_PRECISION}f")}')